# Preprocesado de datos
## Autor: Juan Carlos Alonso Valenzuela
En este notebook se preprocesarán los datos extraídos de la API de Twitter para expresarlos en un formato que pueda ser leído como un grafo.

In [1]:
import pandas as pd
import re
from ast import literal_eval

# Carga de datos

Comenzamos leyendo los archivos

In [2]:
# Neutral
realMadridEspanyol = pd.read_csv('dataset/RealMadridEspanyol.csv')
# Madrid
CAMPEON35 = pd.read_csv('dataset/CAMPEON35.csv')
RMFans = pd.read_csv('dataset/RMFans.csv')
# Espanyol
RCDE = pd.read_csv('dataset/RCDE.csv')
planterRCDE = pd.read_csv('dataset/planterRCDE.csv')

Unimos todos los DataFrames en una sola variable

In [3]:
df = pd.concat([realMadridEspanyol,CAMPEON35,RMFans,RCDE,planterRCDE])
# Shuffle data
df = df.sample(frac=1.)

# Drop Unnamed column
df = df.drop(columns='Unnamed: 0')

# Reset index
df = df.reset_index(drop=True)

In [4]:
df.head()

,username,tweet_content,n_favorites,timestamp,mentions
0,chanclaperica,RT @CasaRonaldBCN: ¡Gracias @pericos_marca por...,0,Tue Apr 26 20:00:03 +0000 2022,"['CasaRonaldBCN', 'pericos_marca']"
1,restebanrf1993,RT @realmadrid: #RMFans x Cibeles x #CAMPEON35...,0,Sat Apr 30 22:55:13 +0000 2022,['realmadrid']
2,mamoschen,RT @realmadrid: 🙌 ¡#CAMPEON35 DE @LALIGA! 🙌\n🏁...,0,Sat Apr 30 16:11:58 +0000 2022,"['realmadrid', 'LaLiga', 'realmadrid', 'RCDEsp..."
3,E4ria_,RT @realmadrid: 🙌 ¡#CAMPEON35 DE @LALIGA! 🙌\n🏁...,0,Sat Apr 30 16:12:05 +0000 2022,"['realmadrid', 'LaLiga', 'realmadrid', 'RCDEsp..."
4,LucasLinero,RT @realmadrid: 🔍 ¿Me podéis decir qué pone en...,0,Sat Apr 30 18:37:34 +0000 2022,['realmadrid']


# Añadir columna "retweet"
Si el tweet es un retweet, esta columna contendrá el nombre del usuario que publicó el tweet original

Un tweet es un retweet si comienza con los caracteres "RT @" seguidos del nombre el usuario al que se está retweeteando.

In [5]:
tweet_content_values = df['tweet_content'].values.tolist()
rt_column = []

for tweet in tweet_content_values:
    rt_regex = re.compile(r"^RT\s@.[^\s]+(?::)")
    
    if rt_regex.match(tweet):
        # The tweet is a retweet
        retweet_from = tweet.split(":")[0][4:]
        rt_column.append(retweet_from)
    else:
        # The tweet is not a retweet
        rt_column.append(None)

df['retweet'] = rt_column

In [6]:
df.head()

,username,tweet_content,n_favorites,timestamp,mentions,retweet
0,chanclaperica,RT @CasaRonaldBCN: ¡Gracias @pericos_marca por...,0,Tue Apr 26 20:00:03 +0000 2022,"['CasaRonaldBCN', 'pericos_marca']",CasaRonaldBCN
1,restebanrf1993,RT @realmadrid: #RMFans x Cibeles x #CAMPEON35...,0,Sat Apr 30 22:55:13 +0000 2022,['realmadrid'],realmadrid
2,mamoschen,RT @realmadrid: 🙌 ¡#CAMPEON35 DE @LALIGA! 🙌\n🏁...,0,Sat Apr 30 16:11:58 +0000 2022,"['realmadrid', 'LaLiga', 'realmadrid', 'RCDEsp...",realmadrid
3,E4ria_,RT @realmadrid: 🙌 ¡#CAMPEON35 DE @LALIGA! 🙌\n🏁...,0,Sat Apr 30 16:12:05 +0000 2022,"['realmadrid', 'LaLiga', 'realmadrid', 'RCDEsp...",realmadrid
4,LucasLinero,RT @realmadrid: 🔍 ¿Me podéis decir qué pone en...,0,Sat Apr 30 18:37:34 +0000 2022,['realmadrid'],realmadrid


In [7]:
df.to_csv('dataset/all_tweets_with_rt.csv')

# Extracción de las aristas del grafo

In [8]:
df_edges = pd.DataFrame({"source": [], "target": [], "weight": []})

In [9]:
for index, row in df.iterrows():
    if row['retweet'] == None:
        username = row['username']
        mentions = literal_eval(row['mentions'])
        if len(mentions)>0:
            # Add every mention to df_edges
            for mention in mentions:
                users = sorted([
                    username,
                    mention
                ])

                new_row = {'source': users[0], 'target': users[1], 'weight':1}
                df_edges = df_edges.append(new_row, ignore_index=True)
    else:
        # Sort alphabetically to avoid duplicates
        users = sorted([
            row['username'],
            row['retweet']
        ])
        # Add to df_edges
        new_row = {'source': users[0], 'target': users[1], 'weight': 1}
        df_edges = df_edges.append(new_row, ignore_index=True)

In [10]:
# Sum by group to avoid duplicates
df_edges = df_edges.groupby(by=['source', 'target']).sum()

In [11]:
df_edges.head()

,,weight
source,target,
0000000000NS,realmadrid,1.0
00900_78601,realmadriden,1.0
00Seergiioo,realmadrid,1.0
012BruceWayne,realmadrid,2.0
01___01a,realmadrid,2.0


In [12]:
# Save as csv
df_edges.to_csv('dataset/dataset_edges.csv', header = False)